# Imports

In [26]:
import math
import json
import os
import glob


import pandas as pd
import screed
# from tqdm import tqdm
# import seaborn as sns

# %matplotlib inline

In [4]:
busco_folder = '/home/olga/data_sm/immune-evolution/databases/busco/orthodb-v10/mammalia_odb10'

In [5]:
ls -lha $busco_folder

total 105M
drwxrwxr-x 5 olga czb 4.0K Jul 15 12:57 ./
drwxrwxr-x 6 olga czb 4.0K Jul 16 08:21 ../
-rwxrwxr-x 1 olga czb 5.7M Nov 20  2019 ancestral*
-rwxrwxr-x 1 olga czb  46M Nov 20  2019 ancestral_variants*
-rwxrwxr-x 1 olga czb  119 Nov 20  2019 dataset.cfg*
drwxrwxr-x 2 olga czb 512K Nov 20  2019 hmms/
drwxrwxr-x 2 olga czb 4.0K Nov 20  2019 info/
-rwxrwxr-x 1 olga czb 256K Nov 20  2019 lengths_cutoff*
-rwxrwxr-x 1 olga czb 855K Nov 27  2019 links_to_ODB10.txt*
-rwxrwxr-x 1 olga czb  52M Jul 15 12:57 mammalia_odb10.fasta*
drwxrwxr-x 2 olga czb 512K Nov 20  2019 prfl/
-rwxrwxr-x 1 olga czb 189K Nov 20  2019 scores_cutoff*


## Read species metadata

In [53]:
species_metadata = pd.read_csv('/home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/species_metadata.csv')
print(species_metadata.shape)
species_metadata.head()

(78, 10)


,proteome_id,tax_id,oscode,n_main_fasta_canonical,n_additional_fasta_isoforms,n_gene2acc,species_name,divergence_from_human_mya,scientific_name,common_name_or_strain
0,UP000007062,7165,ANOGA,12553,971,13619,Anopheles gambiae (African malaria mosquito),797.0,Anopheles gambiae,African malaria mosquito
1,UP000000798,224324,AQUAE,1553,0,1557,Aquifex aeolicus (strain VF5),4290.0,Aquifex aeolicus,strain VF5
2,UP000006548,3702,ARATH,27475,14123,41920,Arabidopsis thaliana (Mouse-ear cress),1496.0,Arabidopsis thaliana,Mouse-ear cress
3,UP000001570,224308,BACSU,4260,7,4268,Bacillus subtilis (strain 168),4290.0,Bacillus subtilis,strain 168
4,UP000001414,226186,BACTN,4782,0,4823,Bacteroides thetaiotaomicron (strain ATCC 2914...,4290.0,Bacteroides thetaiotaomicron,strain ATCC 29148 / DSM 2079 / NCTC 10582 / E5...


In [54]:
species_metadata.query('divergence_from_human_mya < 200')

,proteome_id,tax_id,oscode,n_main_fasta_canonical,n_additional_fasta_isoforms,n_gene2acc,species_name,divergence_from_human_mya,scientific_name,common_name_or_strain
6,UP000009136,9913,BOVIN,23774,14534,38438,Bos taurus (Bovine),96.00,Bos taurus,Bovine
11,UP000002254,9615,CANLF,20271,5269,25631,Canis lupus familiaris (Dog) (Canis familiaris),96.00,Canis lupus familiaris,Dog
28,UP000001519,9595,GORGO,21795,22932,44943,Gorilla gorilla gorilla (Western lowland gorilla),9.06,Gorilla gorilla gorilla,Western lowland gorilla
32,UP000005640,9606,HUMAN,21146,74769,96332,Homo sapiens (Human),0.00,Homo sapiens,Human
40,UP000002280,13616,MONDO,21276,968,22319,Monodelphis domestica (Gray short-tailed opossum),159.00,Monodelphis domestica,Gray short-tailed opossum
42,UP000000589,10090,MOUSE,22287,40369,63134,Mus musculus (Mouse),90.00,Mus musculus,Mouse
52,UP000002277,9598,PANTR,23006,25788,49266,Pan troglodytes (Chimpanzee),6.70,Pan troglodytes,Chimpanzee
60,UP000002494,10116,RAT,21678,9885,32629,Rattus norvegicus (Rat),90.00,Rattus norvegicus,Rat


# Read Busco info

In [6]:
ls -lha $busco_folder/info

total 5.9M
drwxrwxr-x 2 olga czb 4.0K Nov 20  2019 ./
drwxrwxr-x 5 olga czb 4.0K Jul 15 12:57 ../
-rwxrwxr-x 1 olga czb 5.9M Nov 20  2019 ogs.id.info*
-rwxrwxr-x 1 olga czb  602 Nov 20  2019 species.info*


In [16]:
# ! head $busco_folder/*

In [8]:
! head $busco_folder/info/*

==> /home/olga/data_sm/immune-evolution/databases/busco/orthodb-v10/mammalia_odb10/info/ogs.id.info <==
34839_0:0032fe	6011at40674
118797_0:00467b	6011at40674
29078_0:0007f1	6011at40674
9986_0:00087d	6011at40674
9837_0:000f1d	6011at40674
9407_0:001980	6011at40674
9365_0:0046e6	6011at40674
10090_0:0016d4	6011at40674
9606_0:000dcb	6011at40674
1026970_0:0019f2	6011at40674

==> /home/olga/data_sm/immune-evolution/databases/busco/orthodb-v10/mammalia_odb10/info/species.info <==
1026970	Nannospalax galili
246437	Tupaia chinensis
9837	Camelus bactrianus
34839	Chinchilla lanigera
42254	Sorex araneus
185453	Chrysochloris asiatica
9691	Panthera pardus
34882	Enhydra lutris
89399	Rhinolophus sinicus
73337	Ceratotherium simum simum


In [12]:
! grep 9606 $busco_folder/info/ogs.id.info | head

9606_0:000dcb	6011at40674
9606_0:00163a	187617at40674
9606_0:0011b6	174356at40674
9606_0:000911	193343at40674
9606_0:0051a5	117389at40674
9606_0:00358a	157281at40674
9606_0:00037e	197526at40674
9606_0:004119	97431at40674
9606_0:003d9b	106599at40674
9606_0:000891	24284at40674
grep: write error: Broken pipe


# Filter human quest for orthologs proteome for only busco mammlia genes

## Read human Orthodb ids from quest for orthologs


Human taxonomy id is 9606

In [21]:
ls -lha /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/*9606*

-rw-r--r-- 1 olga czb  36M Apr 12  2019 /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000005640_9606_additional.fasta
-rw-r--r-- 1 olga czb  49M Aug 27 08:48 /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000005640_9606__combined_with_additional.fasta
-rw-r--r-- 1 olga czb  34M Apr 13  2019 /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000005640_9606_DNA.fasta
-rw-r--r-- 1 olga czb  20K Apr 13  2019 /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000005640_9606_DNA.miss
-rw-r--r-- 1 olga czb  14M Apr 12  2019 /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000005640_9606.fasta
-rw-r--r-- 1 olga czb  50K May  6  2019 /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000005640_9606_gcoord.miss
-rw-r--r-- 1 olga czb 964M May  6  2019 /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000005640_9606_gcoord.xml


In [47]:

qfo_eukaryota = '/home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota'

def read_ids_filter_orthodb(filename):
    df = pd.read_csv(filename, sep='\t', header=None, names=['uniprot_id', 'id_type', 'db_id'])
    df.columns = 'source__' + df.columns

    df = df.query('source__id_type == "OrthoDB"')
    return df
    
human_ids_orthodb = read_ids_filter_orthodb(f'{qfo_eukaryota}/UP000005640_9606.idmapping')
print(human_ids_orthodb.shape)
human_ids_orthodb.head()

(22539, 3)


,source__uniprot_id,source__id_type,source__db_id
102,P61981,OrthoDB,1176818at2759
200,P27348,OrthoDB,1176818at2759
300,P30443,OrthoDB,1390181at2759
357,Q96QU6,OrthoDB,1156861at2759
786,P10321,OrthoDB,1390181at2759


### Subset human uniprot ids to only busco mammalia ids

In [45]:
busco_mammlia_orthogroups = pd.read_csv(f'{busco_folder}/info/ogs.id.info', sep='\t', header=None, names=['busco_id', 'orthodb_id'])
busco_mammlia_orthogroups.head()

,busco_id,orthodb_id
0,34839_0:0032fe,6011at40674
1,118797_0:00467b,6011at40674
2,29078_0:0007f1,6011at40674
3,9986_0:00087d,6011at40674
4,9837_0:000f1d,6011at40674


In [51]:
set(human_ids_orthodb.source__db_id.values).intersection(busco_mammlia_orthogroups.orthodb_id.values)

set()

In [49]:
human_ids_orthodb_busco = human_ids_orthodb.query('source__db_id in @busco_mammlia_orthogroups.orthodb_id.values')
print(human_ids_orthodb_busco.shape)
human_ids_orthodb_busco.head()

(0, 3)


,source__uniprot_id,source__id_type,source__db_id


## Subset human fasta for only BUSCO mammalia ids

In [34]:
human_fasta = f'{qfo_eukaryota}/UP000005640_9606.fasta'
! head $human_fasta

>tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isoform CRA_a OS=Homo sapiens OX=9606 GN=hCG_2014768 PE=4 SV=1
MSSHEGGKKKALKQPKKQAKEMDEEEKAFKQKQKEEQKKLEVLKAKVVGKGPLATGGIKK
SGKK
>sp|A0A024RBG1|NUD4B_HUMAN Diphosphoinositol polyphosphate phosphohydrolase NUDT4B OS=Homo sapiens OX=9606 GN=NUDT4B PE=3 SV=1
MMKFKPNQTRTYDREGFKKRAACLCFRSEQEDEVLLVSSSRYPDQWIVPGGGMEPEEEPG
GAAVREVYEEAGVKGKLGRLLGIFEQNQDRKHRTYVYVLTVTEILEDWEDSVNIGRKREW
FKVEDAIKVLQCHKPVHAEYLEKLKLGCSPANGNSTVPSLPDNNALFVTAAQTSGLPSSV
R
>tr|A0A075B6H5|A0A075B6H5_HUMAN T cell receptor beta variable 20/OR9-2 (non-functional) (Fragment) OS=Homo sapiens OX=9606 GN=TRBV20OR9-2 PE=4 SV=1
METVVTTLPREGGVGPSRKMLLLLLLLGPGSGLSAVVSQHPSRVICKSGTSVNIECRSLD


In [35]:
! grep 'sp|' $human_fasta | head

>sp|A0A024RBG1|NUD4B_HUMAN Diphosphoinositol polyphosphate phosphohydrolase NUDT4B OS=Homo sapiens OX=9606 GN=NUDT4B PE=3 SV=1
>sp|A0A075B6H9|LV469_HUMAN Immunoglobulin lambda variable 4-69 OS=Homo sapiens OX=9606 GN=IGLV4-69 PE=1 SV=1
>sp|A0A075B6I0|LV861_HUMAN Immunoglobulin lambda variable 8-61 OS=Homo sapiens OX=9606 GN=IGLV8-61 PE=3 SV=7
>sp|A0A075B6I1|LV460_HUMAN Immunoglobulin lambda variable 4-60 OS=Homo sapiens OX=9606 GN=IGLV4-60 PE=3 SV=1
>sp|A0A075B6I4|LVX54_HUMAN Immunoglobulin lambda variable 10-54 OS=Homo sapiens OX=9606 GN=IGLV10-54 PE=3 SV=1
>sp|A0A075B6I9|LV746_HUMAN Immunoglobulin lambda variable 7-46 OS=Homo sapiens OX=9606 GN=IGLV7-46 PE=3 SV=4
>sp|A0A075B6J1|LV537_HUMAN Immunoglobulin lambda variable 5-37 OS=Homo sapiens OX=9606 GN=IGLV5-37 PE=3 SV=1
>sp|A0A075B6J6|LV322_HUMAN Immunoglobulin lambda variable 3-22 OS=Homo sapiens OX=9606 GN=IGLV3-22 PE=3 SV=1
>sp|A0A075B6J9|LV218_HUMAN Immunoglobulin lambda variable 2-18 OS=Homo sapiens OX=9606 GN=IGLV2-18 PE=3 SV=2

In [31]:
! grep P61981 $fasta

In [41]:
human_ids_busco.head()

,source__uniprot_id,source__id_type,source__db_id
102,P61981,OrthoDB,1176818at2759
200,P27348,OrthoDB,1176818at2759
300,P30443,OrthoDB,1390181at2759
357,Q96QU6,OrthoDB,1156861at2759
786,P10321,OrthoDB,1390181at2759


In [42]:
human_ids_busco.shape

(22539, 3)

In [43]:
human_ids_busco.to_csv('/mnt/ibm_sm/home/olga/pipeline-results/human-simulated/busco_mammalia_human_uniprot_ids.csv', index=False)


### How many fasta entries in the human proteome fasta?

In [52]:
! grep -c '>' $human_fasta

20874


In [36]:
def subset_by_uniprot_ids(fasta, uniprot_ids):
    records_subset = []
    with screed.open(fasta) as records:
        for record in records:
            name = record['name']
            record_id = name.split()[0]
            uniprot_id = record_id.split('|')[1]
            if uniprot_id in uniprot_ids:
                records_subset.append(record)
    return records_subset

            
def write_fasta(output_fasta, records):
    with open(output_fasta, 'w') as f:
        for record in records:
            f.write(">{name}\n{sequence}\n".format(**record))

        
# human fasta = protein sequences from quest for orthologs
# human_ids_busco = 
human_busco_mammalia_records = subset_by_uniprot_ids(human_fasta, set(human_ids_busco.source__uniprot_id.values))
print(len(human_busco_mammalia_records))
human_busco_mammalia_records[:3]

18584


[{'name': 'sp|A0A024RBG1|NUD4B_HUMAN Diphosphoinositol polyphosphate phosphohydrolase NUDT4B OS=Homo sapiens OX=9606 GN=NUDT4B PE=3 SV=1', 'sequence': 'MMKFKPNQTRTYDREGFKKRAACLCFRSEQEDEVLLVSSSRYPDQWIVPGGGMEPEEEPGGAAVREVYEEAGVKGKLGRLLGIFEQNQDRKHRTYVYVLTVTEILEDWEDSVNIGRKREWFKVEDAIKVLQCHKPVHAEYLEKLKLGCSPANGNSTVPSLPDNNALFVTAAQTSGLPSSVR', 'description': ''},
 {'name': 'sp|A0A075B759|PAL4E_HUMAN Peptidyl-prolyl cis-trans isomerase A-like 4E OS=Homo sapiens OX=9606 GN=PPIAL4E PE=3 SV=1', 'sequence': 'MVNSVVFFEITRDGKPLGRISIKLFADKIPKTAENFRALSTGEKGFRYKGSCFHRIIPGFMCQGGDFTRPNGTGDKSIYGEKFDDENLIRKHTGSGILSMANAGPNTNGSQFFICAAKTEWLDGKHVAFGKVKERVNIVEAMEHFGYRNSKTSKKITIADCGQF', 'description': ''},
 {'name': 'sp|A0A075B767|PAL4H_HUMAN Peptidyl-prolyl cis-trans isomerase A-like 4H OS=Homo sapiens OX=9606 GN=PPIAL4H PE=3 SV=1', 'sequence': 'MVNSVVFFDITVDGKPLGRISIKLFADKIPKTAENFRALSTGEKGFRYKGSCFHRIIPGFMCQGGDFTRPNGTDDKSIYGEKFDDENLIRKHTGSGILSMVNAGPNTNGSQLFICTAKTEWLDGKHVAFGKVKERVNIVEAMEHFGYRNSKTSKKITIADCGQF', 'des

In [37]:
write_fasta('/home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000005640_9606__busco_mammlia_odbv10.fasta', 
            human_busco_mammalia_records)

## Read mouse busco ids from qfo
Mouse taxononmy id is `10090`

In [17]:
ls -lha /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/*10090*

-rw-r--r-- 1 olga czb  21M Apr 11  2019 /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000000589_10090_additional.fasta
-rw-r--r-- 1 olga czb  35M Apr 13  2019 /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000000589_10090_DNA.fasta
-rw-r--r-- 1 olga czb 5.7K Apr 13  2019 /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000000589_10090_DNA.miss
-rw-r--r-- 1 olga czb  14M Apr 11  2019 /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000000589_10090.fasta
-rw-r--r-- 1 olga czb 559M May  6  2019 /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000000589_10090_gcoord.xml
-rw-r--r-- 1 olga czb 1.9M Apr 11  2019 /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000000589_10090.gene2acc
-rw-r--r-- 1 olga czb  43M May  3  2019 /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000000589_10090.idmapping
-rw-r--r-- 1 olga 

In [40]:
! head /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000000589_10090_DNA.fasta

>tr|A0A075B5I0|ENSMUSP00000093010
TCTCTCACAATAATAGTCAGCAGTGTCCTCAGATGTCAGGCTTCTGAGTTGCATGTAGGC
TGTGTTGGAGGATGTGTCTACAGTAAATGTGGCCTTGCCTTTGAACTTCTGATTGTAGGT
AATACCACCATTTTCAGGATGAATAGCTCCAATCCATTCCAGTCCCTGCTTCAGACTCTG
CTTCACCCATTGAATTTCATAGTCATTAAAATTGTAGCCAGAAGCCTTGCAGGACAACTT
CACCTAAGCCCCAGGTATTACCAGCTCTGGTCCAGACTGCTGAAGTTGGACCTG
>tr|A0A075B5I2|ENSMUSP00000144909
ATGGGCTGTAGGCTCCTAAGCTGTGTGGCCTTCTGCCTCTTGGGAATAGGCCCTTTGGAG
ACGGCTGTTTTCCAGACTCCAAACTATCATGTCACACAGGTGGGAAATGAAGTGTCTTTC
AATTGTAAGCAAACTCTGGGCCACGATACTATGTATTGGTACAAGCAAGACTCTAAGAAA


In [20]:
mouse_ids_busco = read_ids_filter_orthodb(f'{qfo_eukaryota}/UP000000589_10090.idmapping')
print(mouse_ids_busco.shape)
mouse_ids_busco.head()

(25554, 3)


,source__uniprot_id,source__id_type,source__db_id
50,Q9CQV8,OrthoDB,1176818at2759
114,O70456,OrthoDB,1176818at2759
210,P63101,OrthoDB,1176818at2759
255,A2AIG8,OrthoDB,1156861at2759
321,Q6ZWR4,OrthoDB,810409at2759


In [38]:
mouse_fasta = f'{qfo_eukaryota}/UP000000589_10090.fasta'
mouse_busco_mammalia_records = subset_by_uniprot_ids(mouse_fasta, set(mouse_ids_busco.source__uniprot_id.values))
print(len(mouse_busco_mammalia_records))
mouse_busco_mammalia_records[:3]

19150


[{'name': 'tr|A0A087WNP5|A0A087WNP5_MOUSE Predicted gene 10778 OS=Mus musculus OX=10090 GN=Gm10778 PE=1 SV=1', 'sequence': 'MDAVTYEDVYVKFTHEEWALLDPSQKSLYKDVMLETCRNLTAIGYKLQDDNIEEHCPSSRGCGRYNIYGYKPCEHKGYGKKQCTFAQKRFLQTYERICSEEKPYECNQCSKAFANFRSLQKHEKNHTREKPYECNHCGKAFPSRSALQIHQRTHTGEKPFDCSECGKAFARHSNLDIHKRTHTGEKPYDCNECGKAFSTRSHLQIHEIIHTGEKPYVCNQCGKAFAYRSNLEAHEKTHTGEKPYECNQCGNVYASRSSLRNHEKHHTGEKLYGCNQCGKAFARRSSLEIHERTHTGEKPYVCNQCCKAFARRSSLQIHERSHTGEKPYECNQCGKAFASRTSLQNHEKHHTGEKPYGCNQCDKAFELKCDLQIHERIHTGEKPYVCNQCGKAFVSRRYLRNHEKHHTLEKLYDCSQCGKSFAIFRYLQKHKKTHTGEKPYKCTHCGKAFAYRSNLYRHERSHTGEKLYECNECGKAFAYHTSLHNHKKCHIGENPYECN', 'description': ''},
 {'name': 'tr|A0A087WNQ7|A0A087WNQ7_MOUSE Small integral membrane protein 10-like 1 OS=Mus musculus OX=10090 GN=Smim10l1 PE=1 SV=1', 'sequence': 'MGPADAPSALALRAVGPAAATPYGVFCKGLSRTLLAFFELAWQLRMNFPYFYIAGSVILNIRLQGFSGNCCVFQASSRLMISLPSPFDY', 'description': ''},
 {'name': 'tr|A0A087WP21|A0A087WP21_MOUSE ABPA12 OS=Mus musculus OX=10090 GN=Scgb1b12 PE=4 SV=1'

In [39]:
write_fasta('/home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000000589_10090__busco_mammlia_odbv10.fasta', 
            mouse_busco_mammalia_records)

In [44]:
! head '/home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019/Eukaryota/UP000000589_10090__busco_mammlia_odbv10.fasta'

>tr|A0A087WNP5|A0A087WNP5_MOUSE Predicted gene 10778 OS=Mus musculus OX=10090 GN=Gm10778 PE=1 SV=1
MDAVTYEDVYVKFTHEEWALLDPSQKSLYKDVMLETCRNLTAIGYKLQDDNIEEHCPSSRGCGRYNIYGYKPCEHKGYGKKQCTFAQKRFLQTYERICSEEKPYECNQCSKAFANFRSLQKHEKNHTREKPYECNHCGKAFPSRSALQIHQRTHTGEKPFDCSECGKAFARHSNLDIHKRTHTGEKPYDCNECGKAFSTRSHLQIHEIIHTGEKPYVCNQCGKAFAYRSNLEAHEKTHTGEKPYECNQCGNVYASRSSLRNHEKHHTGEKLYGCNQCGKAFARRSSLEIHERTHTGEKPYVCNQCCKAFARRSSLQIHERSHTGEKPYECNQCGKAFASRTSLQNHEKHHTGEKPYGCNQCDKAFELKCDLQIHERIHTGEKPYVCNQCGKAFVSRRYLRNHEKHHTLEKLYDCSQCGKSFAIFRYLQKHKKTHTGEKPYKCTHCGKAFAYRSNLYRHERSHTGEKLYECNECGKAFAYHTSLHNHKKCHIGENPYECN
>tr|A0A087WNQ7|A0A087WNQ7_MOUSE Small integral membrane protein 10-like 1 OS=Mus musculus OX=10090 GN=Smim10l1 PE=1 SV=1
MGPADAPSALALRAVGPAAATPYGVFCKGLSRTLLAFFELAWQLRMNFPYFYIAGSVILNIRLQGFSGNCCVFQASSRLMISLPSPFDY
>tr|A0A087WP21|A0A087WP21_MOUSE ABPA12 OS=Mus musculus OX=10090 GN=Scgb1b12 PE=4 SV=1
MKLAGAMVILGAALLLLTSGGDCGICPAIKEDVRLFLNGTSEAYVEYVKQYKDDPVTLENTAKIKQCVDSTLTEEDRAHATTFIEKIEASPLC
>tr|A0A087